# Face Detection and Recognition using YOLOv8 and FaceNet

In [ ]:
# Step 1: Install Required Packages
!pip install ultralytics keras mtcnn opencv-python tensorflow scikit-learn matplotlib imutils

In [ ]:
# Step 2: Face Detection using YOLOv8
from ultralytics import YOLO
import cv2

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

def detect_faces(image):
    results = model(image)
    faces = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            face = image[y1:y2, x1:x2]
            faces.append(face)
    return faces

In [ ]:
# Step 3: Load FaceNet and Generate Embeddings
from keras.models import load_model
import numpy as np
from mtcnn.mtcnn import MTCNN

assert os.path.exists('facenet_keras.h5'), 'Download facenet_keras.h5 and place it in the working directory.'
facenet_model = load_model('facenet_keras.h5')

def preprocess_face(face, required_size=(160, 160)):
    face = cv2.resize(face, required_size)
    face = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    return np.expand_dims(face, axis=0)

def get_embedding(model, face):
    face = preprocess_face(face)
    embedding = model.predict(face)
    return embedding[0]

In [ ]:
# Step 4: Build Embedding Database
import os

embedding_db = {}
dataset_path = 'dataset'

for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    embeddings = []
    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        image = cv2.imread(img_path)
        faces = detect_faces(image)
        if faces:
            emb = get_embedding(facenet_model, faces[0])
            embeddings.append(emb)
    if embeddings:
        embedding_db[person] = np.mean(embeddings, axis=0)

In [ ]:
# Step 5: Face Recognition
from numpy.linalg import norm

def recognize_face(face, database, threshold=0.7):
    emb = get_embedding(facenet_model, face)
    min_dist = 100
    identity = None
    for name, db_emb in database.items():
        dist = norm(emb - db_emb)
        if dist < min_dist:
            min_dist = dist
            identity = name
    if min_dist < threshold:
        return identity, min_dist
    else:
        return 'Unknown', min_dist